In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import cv2
import os 
import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image
import random
import matplotlib.image as mpimg
from imutils import paths
import sys
import operator

In [ ]:
!pip install rich
!pip install hydra-core
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.4 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 4.3 MB/s 
     |████████████████████████████████| 117 kB 54.9 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=afbe5798d96b3586cad5cd268849c423564ce5f3ee604650cbbdee4bfd2abebc
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 706 kB 4.1 MB/s 
     |████████████████████████████████| 419 kB 50.6 MB/s 
     |████████████████████████████████| 5.9 MB 48.0 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


In [ ]:
sys.path.insert(0, "/content/drive/MyDrive/Github_Repos/historia_ml_handwritten_text_line_segmentation")
sys.path.insert(1, "/content/drive/MyDrive/Github_Repos/historia_ml_handwritten_text_line_segmentation/src/utils")

In [ ]:
import pytorch_lightning as pl
from src.utils.display_image_from_path import img_show
from src.utils.display_image_from_array import img_show_from_arr
from src.utils.delete_border_lines import delete_border_lines
from src.utils.take_pixels_whose_neighbors_are_zero import next_neighbor_as_zero
from src.utils.get_row_heights import get_row_heights
from src.utils.get_col_widths import get_col_widths
from src.utils.erase_short_lines import erase_lines_with_small_width
from src.utils.erase_small_lines import erase_lines_with_small_height
from src.utils.find_image_page_type import find_image_page_type
from src.utils.create_row_patches_from_lines_y_coordinates import create_row_patches
from src.utils.get_unique_cols_from_row_patches import create_unique_patch_cols
from src.utils.create_row_patch_mapping import get_patch_map
from src.utils.create_bboxes import create_bbox_list
from src.utils.create_line_images_from_bbox import create_line_image_from_bbox
from src.utils.create_row_images_from_bbox_list import create_row_images
from src.utils.get_bboxes_from_mask import get_bboxes_from_mask
from src.utils.save_line_images import save_line_images
from src.utils.save_line_images_v2 import save_line_images_v2
from src.utils.display_and_save_line_images import display_and_save_line_images
from src.utils.calculate_the_safety_gaps import calculate_safety_gap_xy
from src.utils.preprocess_img_mask import preprocess_erasing

In [14]:
img_path = "/content/drive/MyDrive/Images/esad-127.png"
img = cv2.imread(img_path)
img_mask = cv2.imread("/content/drive/MyDrive/output/test_page_predicted_masks/esad-127_mask_310x310.png")

img_show_from_arr(img)
img_show_from_arr(img_mask)

In [15]:
cfg = {
    "img_path": img_path,
    "x_ratio": 0.07,
    "y_ratio": 0.1,
    "col_width_limit_ratio": 0.7,
    "row_height_limit_ratio": 0.3,
    "line_gap_ratio": 0.05,
    "safety_gap_x_ratio": 0.08,
    "safety_gap_y_ratio": 0.5,
    "safety_erase_ratio_width": 0.01,
    "safety_erase_ratio_height": 0.01,
    "patch_unique_col_size_limit_ratio": 0.5,
    "lower_freq_y_ratio": 0.6,
    "higher_freq_y_ratio": 1.4,
    "lower_freq_x_ratio":0.5
}

In [ ]:
mask = img_mask.copy()
mask = preprocess_erasing(mask, cfg["x_ratio"], cfg["y_ratio"], cfg["lower_freq_y_ratio"], cfg["higher_freq_y_ratio"], cfg["lower_freq_x_ratio"])
img_show_from_arr(img_mask)
img_show_from_arr(mask)

In [ ]:
bboxes, page_type = get_bboxes_from_mask(mask=mask,
                              col_width_limit_ratio=cfg["col_width_limit_ratio"],
                              row_height_limit_ratio=cfg["row_height_limit_ratio"],
                              safety_erase_ratio_width=cfg["safety_erase_ratio_width"],
                              safety_erase_ratio_height=cfg["safety_erase_ratio_height"],
                              line_gap_ratio=cfg["line_gap_ratio"],
                              patch_unique_col_size_limit_ratio=cfg["patch_unique_col_size_limit_ratio"])
len(bboxes)

In [ ]:
bboxes

In [ ]:
len(bboxes)

In [ ]:
safety_gap_xy = calculate_safety_gap_xy(bboxes, cfg["safety_gap_x_ratio"], cfg["safety_gap_y_ratio"])

In [ ]:
display_and_save_line_images(bboxes, 
                             img,
                             safety_gap_xy[0],
                             safety_gap_xy[1],
                             page_type,
                             save=True,
                             img_path=img_path)